In [91]:
import torch
from torch import nn
from circuits import Circuit,Kinds
from learn import Trainer
from models import Solver
from models import State
from data import Preprocess

In [92]:
# Get cpu or gpu device for training.
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

# torch.autograd.set_detect_anomaly(True)
torch.set_printoptions(precision=3, linewidth=200)

Using cpu device


In [93]:
vr = Circuit()
vr.ring(Kinds.IVS,Kinds.R,3)
vr.elements[0].attr = 1000
vr.elements[-1].v = 1

In [94]:
input = Preprocess(vr)
model = Solver(input=input, state=State.Solve)
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.1)
trainer = Trainer(model,optimizer,nn.MSELoss())
loss, _ = trainer.train()
print(f'init target: {model.input.target}')
print(f'init params: {model.attr}')
print(f'init loss: {loss.item()}')

1
1000
1000
init target: [1.0, 1.0, 1.0, 1.0, 1.0, 0.001, 0.001, 0.001, 1.0, 1.0, 1.0, 1.0]
init params: Parameter containing:
tensor([1.000e+00, 1.010e-01, 1.010e-01, 1.000e-12], requires_grad=True)
init loss: 0.055222731083631516


In [95]:
num_epochs = 1000
prev_loss = 0.1

epoch = 0
while(epoch < num_epochs):
    loss, _ = trainer.train()
    if(loss < 1e-15):
            break
    epoch += 1
    loss_change = abs(loss - prev_loss) / prev_loss
    prev_loss = loss

print(f'Done! at {epoch} passes, {loss.item()} loss')
solution = model()
i_norm, v_norm = model.split_solution(solution)
print('norm i')
print(i_norm)
print('norm v')
print(v_norm)
i_real, v_real = model.denorm_solution(solution)
print('real i')
print(i_real)
print('real v')
print(v_real)
print('norm attr')
print(model.attr)
print('real attributes')
print(model.attr*model.r_base_from_i_v())

Done! at 531 passes, 9.478366442411401e-16 loss
norm i
tensor([[-0.830],
        [ 0.830],
        [ 0.830],
        [ 0.830]], grad_fn=<SliceBackward0>)
norm v
tensor([[1.000e+00],
        [4.995e-01],
        [4.995e-01],
        [1.000e-03]], grad_fn=<SliceBackward0>)
real i
tensor([[-0.830],
        [ 0.830],
        [ 0.830],
        [ 0.830]], grad_fn=<MulBackward0>)
real v
tensor([[1.000e+03],
        [4.995e+02],
        [4.995e+02],
        [1.000e+00]], grad_fn=<MulBackward0>)
norm attr
Parameter containing:
tensor([1.000, 0.602, 0.602, 0.001], requires_grad=True)
real attributes
tensor([1000.000,  601.550,  601.550,    1.204], grad_fn=<MulBackward0>)
